## Escalar variables numèriques dels df

FALTA GUARDAR ELS DATAFRAMES NORMALITZATS

In [7]:
import pyspark
from pyspark.sql import SparkSession
import logging
import duckdb

logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("pyspark").setLevel(logging.ERROR)
logging.getLogger("spark").setLevel(logging.ERROR)
logging.getLogger("duckdb").setLevel(logging.ERROR)

!curl -o duckdb.jar "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"

sc = SparkSession.builder.config("spark.jars", "duckdb.jar").getOrCreate()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61.0M  100 61.0M    0     0  37.2M      0  0:00:01  0:00:01 --:--:-- 37.2M--:--:--  0:05:29  188k


In [12]:
conn = duckdb.connect("compravenda_sup.db")
conn.close()

In [8]:
paths = ['data_lake/compravenda_sup.parquet', 'data_lake/rent_price.parquet', 'data_lake/renda.parquet']
col_numeriques = {'data_lake/compravenda_sup.parquet': ['Nombre']
                  , 'data_lake/renda.parquet': ['Import_Euros']
                , 'data_lake/rent_price.parquet': ['Price']}

In [9]:
for path in paths:
    RDD = sc.read.parquet(path)
    for columna in col_numeriques[path]:
        valor_min = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
            .map(lambda x: (1, x[columna]))\
            .mapValues(lambda x: float(x) if type(x) == str else x)\
            .reduceByKey(lambda x, y: min(x,y)).collect()[0][1]
        valor_max = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
            .map(lambda x: (1, float(x[columna])))\
            .reduceByKey(lambda x, y: max(x,y)).collect()[0][1]
        
        RDD = RDD.withColumn(columna, (RDD[columna] - valor_min)/ (valor_max - valor_min))

    # falta el RDD.write

In [11]:
sc.stop()